In [1]:
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()

In [2]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt

In [3]:
from ftfy import fix_text
import threading
import os

filename = 'data/fulldataset.csv'
sample_frac = 0.1

df = pd.read_csv(filename,quotechar="\"")
df = df.sample(frac=sample_frac, random_state=1)
df.reset_index(inplace=True, drop=True)
num_rows = len(df.index)
print("num rows", num_rows)

num rows 51617


In [12]:
# sanity check cell, skip this if training on full data
df = pd.read_csv('sanity_check.csv')
sample_frac = 1.0
df

,Band,Lyrics,Song
0,Mariah Carry,I don't want a lot for Christmas\nThere is jus...,All I Want for Christmas is You
1,Wham!,Last Christmas I gave you my heart\nBut the ve...,Last Christmas
2,Idk,"Jingle bells, jingle bells, jingle all the way...",Jingle Bells
3,Johnny Mathis,"Jingle bell, jingle bell Jingle bell rock Jing...",Jingle Bell Rock
4,Lee Greenwood,If tomorrow all the things were gone\nI worked...,God Bless the USA
5,Samuel Ward,Oh beautiful for heroes proved\nIn liberating ...,America the Beautiful
6,Francis Scott Key,"Oh, say can you see by the dawn’s early light\...",Star Spangled Banner


In [5]:
pickle_filename = f"data/lyrec_df_{sample_frac}.pkl"
if os.path.isfile(pickle_filename):
    print("Found pickle file for current config, using that")
    df = pd.read_pickle(pickle_filename)
else:
    print("Cleaning dataset, no cache available")
    pbar = tqdm(total=(num_rows))
    def clean(df, row):
        try:
            lyric = fix_text(df['Lyrics'][row])
            band = fix_text(df['Band'][row])
            song = fix_text(df['Song'][row])
            df['Lyrics'][row] = lyric
            df['Band'][row] = band
            df['Song'][row] = song
        except Exception:
            print("Error cleaning up row:", row,"dropping instead")
            df.drop(index=row)
        pbar.update(1)

    threads = []
    for row in df.index:
        t = threading.Thread(target=clean, args=(df, row))
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

df.reset_index(inplace=True, drop=True)
df

Found pickle file for current config, using that


,Band,Lyrics,Song
0,Reba McEntire,The greatest man I never knew lived just down ...,The Greatest Man I Never Knew
1,Vanessa Carlton,Stealing glances through the key hole\nIn a br...,Hear the Bells
2,Snoop Dogg,"What it do,\nComing at you live, it's your boy...",Pronto
3,Snoop Dogg,Another smoke session up in this motherfucker\...,Buss 'N Rocks
4,The Shirelles,"Everybody, twist\n\nThey're twisting in Clevel...",Twisting USA
...,...,...,...
51612,Litfiba,"---------\nWoda woda, woda woda\nWoda woda, wo...",Woda Woda
51613,Robbie Williams,You're one of God's better people\nAnd you don...,One of God's Better People
51614,The Vipers,There is no greater love\nThan what I feel for...,There Is No Greater Love
51615,The Golden Palominos,It's feeling like the same old dance\nWe never...,Gun


In [6]:
# save dataset to pickle
df.to_pickle(pickle_filename)

## Attempt at BERT pairwise similarities

In [13]:
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
document_embeddings = None
got_from_file = False
filepath = f"data/lyrec_embeddings_{sample_frac}.pkl"

if os.path.isfile(filepath):
    got_from_file = True
    with open(filepath, 'rb') as f:
        document_embeddings = np.load(f)
else:
    document_embeddings = sbert_model.encode(df['Lyrics'], show_progress_bar=True, device="cpu")

if not got_from_file:
    with open(filepath, 'wb') as f:
        np.save(document_embeddings, filepath)      

In [15]:
from sklearn.metrics import pairwise
print(document_embeddings.shape)
pairwise_similarities=pairwise.cosine_similarity(document_embeddings)
pairwise_differences=pairwise.euclidean_distances(document_embeddings)

from sklearn.feature_extraction.text import TfidfVectorizer 

tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(df.Lyrics)
tfidf_vectors=tfidfvectoriser.transform(df.Lyrics)

def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Song: {df.iloc[doc_id]["Song"]} - {df.iloc[doc_id]["Band"]}')
    print ('\n')
    print ('Similar Songs:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix[:5]:
#         if ix==doc_id:
#             continue
        print('\n')
        print (f'Similar Song: {df.iloc[ix]["Song"]} - {df.iloc[ix]["Band"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

for i in range(7):
    most_similar(i,pairwise_similarities,'Cosine Similarity')
    print("---")


(7, 768)
Song: All I Want for Christmas is You - Mariah Carry


Similar Songs:


Similar Song: All I Want for Christmas is You - Mariah Carry
Cosine Similarity : 1.0


Similar Song: Last Christmas - Wham!
Cosine Similarity : 0.7781903147697449


Similar Song: God Bless the USA - Lee Greenwood
Cosine Similarity : 0.568848729133606


Similar Song: America the Beautiful -  Samuel Ward
Cosine Similarity : 0.5428792238235474


Similar Song: Jingle Bell Rock - Johnny Mathis
Cosine Similarity : 0.5265178680419922
---
Song: Last Christmas - Wham!


Similar Songs:


Similar Song: Last Christmas - Wham!
Cosine Similarity : 1.0


Similar Song: All I Want for Christmas is You - Mariah Carry
Cosine Similarity : 0.7781903147697449


Similar Song: God Bless the USA - Lee Greenwood
Cosine Similarity : 0.6534405946731567


Similar Song: America the Beautiful -  Samuel Ward
Cosine Similarity : 0.6120215654373169


Similar Song: Jingle Bell Rock - Johnny Mathis
Cosine Similarity : 0.5684051513671875
---


## Bag of Words

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import re
from operator import itemgetter
from sklearn.metrics import pairwise
# Define dictionary utility function
BAG_SIZE = 100
def get_top_values(d,N = BAG_SIZE):
    return list(sorted(d.items(), key = itemgetter(1), reverse = True)[:N])

# Preprocess songs

# set stop words in english
stop_words = set(stopwords.words('english')) 

# creating  "master" dictionary
word2count = {}

for row in df.index:
    song = df['Lyrics'][row]

    # remove stopwords and clean up each song
    song = song.lower()
    song = re.sub(r'\W', ' ', song)
    song = re.sub(r'\s+', ' ', song)
    song = re.sub(r'\r|\n', ' ', song)

    words = nltk.word_tokenize(song)
    for word in words:

        if word not in stop_words:

            if word not in word2count.keys():
                word2count[word] = 1
            else:
                word2count[word] += 1

word2count_list = get_top_values(word2count)
word2count = dict(word2count_list)
word_ranks = {w2c[0]: r for r, w2c in enumerate(word2count_list)}

# Now pass through again and create vectors

song_counts = np.zeros([len(df.index), BAG_SIZE])

for song_index in df.index:

    song = df['Lyrics'][song_index]

    # remove stopwords and clean up each song
    song = song.lower()
    song = re.sub(r'\W', ' ', song)
    song = re.sub(r'\s+', ' ', song)
    song = re.sub(r'\r|\n', ' ', song)

    words = nltk.word_tokenize(song)
    for word in words:
        if word in word_ranks:
            word_idx = word_ranks[word]
            song_counts[song_index][word_idx] += 1

print(word2count)
print(song_counts)


pairwise_similarities=pairwise.cosine_similarity(song_counts)
pairwise_differences=pairwise.euclidean_distances(song_counts)


def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Song: {df.iloc[doc_id]["Song"]} - {df.iloc[doc_id]["Band"]}')
    print ('Similar Songs:')
    
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix[:5]:
#         if ix==doc_id:
#             continue
        print('\n')
        print (f'Similar Song: {df.iloc[ix]["Song"]} - {df.iloc[ix]["Band"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

for i in range(7):
    most_similar(i,pairwise_similarities,'Cosine Similarity')
    print("---")
    #most_similar(0,pairwise_differences,'Euclidean Distance')


{'jingle': 49, 'christmas': 23, 'oh': 21, 'bells': 20, 'gave': 20, 'bell': 19, 'want': 16, 'one': 15, 'baby': 14, 'horse': 14, 'sleigh': 13, 'special': 13, 'away': 12, 'know': 11, 'fun': 11, 'next': 10, 'time': 10, 'heart': 9, 'someone': 9, 'god': 9, 'way': 9, 'ride': 9, 'open': 9, 'year': 8, 'give': 8, 'land': 8, 'america': 8, 'day': 7, 'cause': 7, 'last': 7, 'save': 7, 'love': 7, 'rock': 7, 'free': 7, 'sea': 7, 'right': 6, 'tears': 6, 'still': 6, 'er': 6, 'home': 6, 'make': 5, 'wish': 5, 'shining': 5, 'bright': 5, 'stand': 5, 'need': 4, 'could': 4, 'ever': 4, 'go': 4, 'ring': 4, 'sing': 4, 'today': 4, 'american': 4, 'star': 4, 'spangled': 4, 'banner': 4, 'wave': 4, 'brave': 4, 'lot': 3, 'underneath': 3, 'come': 3, 'true': 3, 'snow': 3, 'tonight': 3, 'air': 3, 'see': 3, 'fool': 3, 'night': 3, 'proud': 3, 'least': 3, 'forget': 3, 'men': 3, 'died': 3, 'gladly': 3, 'defend': 3, 'doubt': 3, 'bless': 3, 'u': 3, 'say': 3, 'may': 3, 'thy': 3, 'shed': 3, 'grace': 3, 'thee': 3, 'lord': 3, 'thi

In [ ]:
# save matrix as pickle
with open(f'data/lyrec_similarity_mat_{sample_frac}.pkl', 'wb') as f:
    np.save(f, pairwise_similarities)